# Cálculo huella carbono

In [2]:
import pandas as pd
from unidecode import unidecode

## Peso y volumen de los libros

El peso de las 500 unidades de libros incluye el peso de un europalet de 25 kg.

In [3]:
libros = pd.read_csv('../data/raw/PesoVolumenLibros.csv', encoding='latin1')
display(libros)

,libro_titulo,libro_isbn,ancho_cm,alto_cm,grosor_cm,paginas,vol_cm3_1libro,vol_m3_500libros,peso_g_1libro,peso_sineuropalet_t_500libros,peso_t_500libros
0,Niebla de sueño,978-84-1353-114-4,13,19,1.2,162,296.4,0.14820,250,0.125,0.150
1,La isla de los canarios. Volumen 6,978-84-1353-115-1,21,23,0.8,92,386.4,0.19320,400,0.200,0.225
2,El mundo del libro en Canarias,84-8103-396-0,15,21,3.1,532,976.5,0.48825,780,0.390,0.415
3,Historia del Cabildo insular de Gran Canaria,84-8103-067-8,17,24,3.9,748,1591.2,0.79560,1240,0.620,0.645
4,Flora de Gran Canaria 1,978-84-8103-741,27,34,3.0,238,2754.0,1.37700,2280,1.140,1.165


## Etapa 1: identificación de los diferentes trayectos de este servicio de transporte

La Ruta es el Camino (a ser) tomado para llegar de un punto de partida a un punto de destino. (AENOR2013)

El Trayecto Para un servicio de transporte de mercancías o de pasajeros, la sección de la ruta tomada o que se ha de tomar, dentro de la cual el vehículo transporta las mercancías o el pasajero (AENOR2013).

In [3]:
salida_carga_distance = pd.read_csv('../data/processed/city_port_distance.csv')
salida_carga_distance.drop(columns=['Unnamed: 0', 'id', 'lat', 'lon'], inplace=True)
# Si el puerto más cercano es el de Las Palmas significa que la ciudad está en 
# Gran Canaria, por lo tanto se irá directamente al punto de despacho sin
# necesidad de ir al puerto (este trayecto será 0).
salida_carga_distance.loc[salida_carga_distance['port_name']=='LAS PALMAS', 'shortest_distance_to_port_km'] = 0 

carga_descarga_distance = pd.read_csv('../data/processed/port_portlp_distance.csv')
carga_descarga_distance.drop(columns='Unnamed: 0', inplace=True)

descarga_llegada_distance = pd.read_csv('../data/processed/puertodescarga_llegada_distance.csv')
descarga_llegada_distance.drop(columns='Unnamed: 0', inplace=True)

salida_llegada_distance = pd.read_csv('../data/processed/cityGC_destination_distance.csv')
salida_llegada_distance.drop(columns='Unnamed: 0', inplace=True)

In [4]:
# Dataframes
salida_carga = salida_carga_distance.copy()

carga_descarga = carga_descarga_distance.copy()

descarga_llegada = descarga_llegada_distance.copy()

salida_llegada = salida_llegada_distance.copy()

## Etapa 2: cálculo del consumo de energía y de emisiones de GEI de cada trayecto

### Subetapa 2.1: establecimiento del sitema de operación de un vehículo (VOS) relativo a este trayecto

#### 1. Trayecto salida-carga

- Trayecto 1 - VOS de ciudad a puerto por carretera
- A 12-24 t lorry transports a container (1 TEU) with average goods. El contenedor pesa 1.95 toneladas, y lleva una carga de 10.5 toneladas, por lo tanto, su peso total es 12.45 toneladas (Table 9, (Schmied2012))
- El contenedor está cargado al 60%. **Factor de carga**. Ratio obtenido por la relación entre la carga real y la carga máxima autorizada de un medio de transporte. (AENOR2013)
- Specific energy consumption (Table 12, (Schimied2012)) = 0.24 liter/TEU-km. Ya considera Additional empty trips +10% (Schmied2012).
- Asumimos que el camión viaja mayoritariamente por autopistas/autovías, por terreno plano. No aplicamos factor de corrección por el uso de carreteras urbanas.
- El motor es de diesel (no biofuel).
- Calculamos sólo ida.

#### 2. Trayecto carga-descarga

- Trayecto 2 - VOS de puerto a Puerto de Las Palmas
- Ruta peninsuar.
    - Realizado siguiendo la metodología de Clean Cargo (CleanCargo2015, CleanCargo2021)
    - No conocemos el tipo de barco que realiza el transporte, por lo que un factor de emisión asociado al tipo de barco no es lo más adecuado. En lugar de esto, conocemos el punto de salida y de llegada, así como la ruta (e.g. Europe (North and Med) to-from Africa). Como resultado, un factor de emisión agregado para la rota es más apropiado. Esto es lo que provee Clean Cargo (CleanCargo2021).
    - Utilization factor: Se aplica un 70% como factor de utilización.
    - Transported distance: Se obtiene de las tablas. La distancia que tenemos es la más corta. CCWG dice que siempre hay desviaciones y que la ruta real es más larga. Los estudios que ha realizado hasta la fecha que aplicando un factor de ajuste para la distancia de un 15% más mejora precisión media de las estimas de emisiones de CO2. En este estudio el factor de corrección se aplicará a los trayectos desde península, pero no entre islas. (Pragmatic solution to a complex issue).
    - Asumimos que es un barco average para la ruta.
    - en la Tabla 37 de (EcoTransIT2023) obtenemos orientación del número de TEUs que transportan los barcos según su línea.
    
- Ruta canaria.
    - Volcán de Tamadaba (https://www.marinetraffic.com/en/ais/details/ships/shipid:168397/mmsi:224957000/imo:9360506/vessel:VOLCAN_DE_TAMADABA). Info: https://www.shipafreight.com/knowledge-series/deadweight-tonnage/. 
    - Default cargo utilisation 70% (Table 39, EcoTransIT2023).
    - Aggregated size class < 5000 DWT (Table 38, EcoTransIT2023). De la web de Marine Traffic Summer DWT: 3400 t.



#### 3. Trayecto descarga-llegada

- Trayecto 3 - VOS de Puerto de Las Palmas a punto de despacho
- Un camión < 7.5 t GVW transporta average goods tiene un maximum payload de 3.5 t y como va al 60% de su capacidad total, la carga que transporta pesa 2.1 t. Additional empty trips = 20% (Schmied2012, Tabla 10)
- Asumimos que el terreno por el que circula es relativamente plano.
- Transporta average goods
- En base a lo anterior, specific energy consumption litre/tkm = 0.077
- Este trayecto se realiza en Gran Canaria (puerto-llegada) y aplicamos un factor de corrección  (Schmied2012, Tabla 11). El factor de corrección es 0.9.
- El camión es de diesel.

#### 4. Trayecto salida-llegada

- Este trayecto corresponde al viaje desde las imprentas en Gran Canaria al punto de destino.
- Asumimos que el camión que realiza este transporte es equivalente al que realiza el trayecto de descarga-llegada.

### Subetapa 2.2: cuantificación del consumo total de combustible para el VOS

- La distancia a cubrir es entre las ciudades de origen en la península y en el resto de islas Canarias (no GC) está en la tabla *salida_carga_distance*.
- Para el cálculo del consumo de combustible utilizaremos el *consumo energético específico* o *specific energy consumption*. Table 12 de (Schimied2012)

#### 1. Trayecto salida-carga

In [5]:
# Definir funcion para el cálculo del consumo de diesel en base a la distancia.
# Se transporta un TEU, por esto no se pone actual cargo weight.
def salida_carga_diesel(distancia_trayecto):
    specific_energy_consumption_lTEUkm = 0.24
    diesel_l = distancia_trayecto * specific_energy_consumption_lTEUkm
    return diesel_l

In [6]:
salida_carga['consumo_diesel_TEU_l'] = salida_carga.apply(lambda row:salida_carga_diesel(row['shortest_distance_to_port_km']), axis=1)

#### 2. Trayecto carga-descarga

##### 1. Trayecto peninsular

##### 2. Trayecto canario

#### 3. Trayecto descarga-llegada

In [7]:
# Definir funcion para el cálculo del consumo de diesel en base a la distancia.
def descarga_llegada_diesel(distancia_trayecto):
    specific_energy_consumption_ltkm = 0.077
    actual_cargo_weight_t = 2.1
    factor_correccion_carreteras_urbanas = 0.9
    diesel_l = (distancia_trayecto * actual_cargo_weight_t * specific_energy_consumption_ltkm) * factor_correccion_carreteras_urbanas
    return diesel_l

In [8]:
descarga_llegada['consumo_diesel_total_l'] = descarga_llegada.apply(lambda row:descarga_llegada_diesel(row['descarga_llegada_distancia_km']), axis=1)

#### 4. Trayecto salida-llegada

In [9]:
salida_llegada['consumo_diesel_total_l'] = salida_llegada.apply(lambda row:descarga_llegada_diesel(row['salida_llegada_distancia_km']), axis=1)

### Subetapa 2.3: cálculo del consumo total de energía y de emisiones de GEI para este VOS

En base al consumo de combustible, se calcula:
1. Consumo de energía de total (MJ). Para esto se multiplica por los factores energéticos (WTW y TTW).
2. Emisiones de carbono (kgCO2/e). Para esto se multiplica por los factores de emisión (WTW y TTW).

#### 1. Trayecto salida-carga

##### Energía total

In [10]:
# Datos (AENOR2013)
def carretera_consumo_energetico_TTW(consumo_diesel):
    factor_energetico_TTW_et_MJl = 35.9
    carretera_TTW_MJ = consumo_diesel * factor_energetico_TTW_et_MJl
    return carretera_TTW_MJ

In [11]:
def carretera_consumo_energetico_WTW(consumo_diesel):
    factor_energetico_WTW_ew_MJl = 42.7
    carretera_WTW_MJ = consumo_diesel * factor_energetico_WTW_ew_MJl
    return carretera_WTW_MJ

In [12]:
def carretera_consumo_energetico_WTT(consumo_diesel):
    wtw = carretera_consumo_energetico_WTW(consumo_diesel)
    ttw = carretera_consumo_energetico_TTW(consumo_diesel)
    wtt = wtw - ttw
    return wtt

##### Emisiones carbono

In [13]:
# Datos (AENOR2013)
def carretera_emisiones_carbono_TTW(consumo_diesel):
    factor_emision_TTW_gt_kgCO2el = 2.67
    carretera_TTW_kgCO2e = consumo_diesel * factor_emision_TTW_gt_kgCO2el
    return carretera_TTW_kgCO2e

In [14]:
def carretera_emisiones_carbono_WTW(consumo_diesel):
    factor_emision_WTW_gw_kgCO2el = 3.24
    carretera_WTW_kgCO2e = consumo_diesel * factor_emision_WTW_gw_kgCO2el
    return carretera_WTW_kgCO2e

In [15]:
def carretera_emisiones_carbono_WTT(consumo_diesel):
    wtw = carretera_emisiones_carbono_WTW(consumo_diesel)
    ttw = carretera_emisiones_carbono_TTW(consumo_diesel)
    wtt = wtw - ttw
    return wtt

In [16]:
salida_carga['emisionesC_TTW_TEU_kgCO2e'] = salida_carga.apply(lambda row:carretera_emisiones_carbono_TTW(row['consumo_diesel_TEU_l']), axis=1)

salida_carga['emisionesC_WTT_TEU_kgCO2e'] = salida_carga.apply(lambda row:carretera_emisiones_carbono_WTT(row['consumo_diesel_TEU_l']), axis=1)

salida_carga['emisionesC_WTW_TEU_kgCO2e'] = salida_carga.apply(lambda row:carretera_emisiones_carbono_WTW(row['consumo_diesel_TEU_l']), axis=1)

#### 2. Trayecto carga-descarga

##### 1. Trayecto peninsular

In [17]:
# Trade Lane: Europe (North and Med) to-from Africa
# Dry (No Reefer)
# 2020 WTW CO2e, 70% UF = 100.2 grams of CO2 per TEU km
# Extra 15% for route deviation
# 1 TEU = 1 tonne

def mar_peninsula_emisiones_carbono_WTW(distancia_trayecto):
    emission_factor_WTW_gCO2TEUkm = 100.2
    detour = distancia_trayecto * 0.15
    teus_shipment = 1 
    mar_WTW_gCO2e = emission_factor_WTW_gCO2TEUkm * (distancia_trayecto + detour) * teus_shipment
    mar_WTW_kgCO2e = mar_WTW_gCO2e/1000
    return mar_WTW_kgCO2e

In [18]:
# 2020 TTW CO2e, 100% UF = 59.7 grams of CO2 per TEU km
# Aplicamos 70% utilization factor
def mar_peninsula_emisiones_carbono_TTW(distancia_trayecto):
    emission_factor_TTW_gCO2TEUkm = 59.7
    detour = distancia_trayecto * 0.15
    teus_shipment = 1
    mar_TTW_gCO2e = (emission_factor_TTW_gCO2TEUkm * (distancia_trayecto + detour) * teus_shipment) / 0.7
    mar_TTW_kgCO2e = mar_TTW_gCO2e/1000
    return mar_TTW_kgCO2e

In [19]:
def mar_peninsula_emisiones_carbono_WTT(distancia_trayecto):
    wtw = mar_peninsula_emisiones_carbono_WTW(distancia_trayecto)
    ttw = mar_peninsula_emisiones_carbono_TTW(distancia_trayecto)
    wtt = wtw - ttw
    return wtt

##### 2. Trayecto canario

In [20]:
# Trade Lane: Fleet-Wide Average CO2 Performance
# Dry (No Reefer)
# 2020 WTW CO2e, 70% UF = 66.4 grams of CO2 per TEU km
# Extra 15% for route deviation
# 1 TEU = 1 tonne

def mar_canarias_emisiones_carbono_WTW(distancia_trayecto):
    emission_factor_WTW_gCO2TEUkm = 110.9
    detour = distancia_trayecto * 0.15
    teus_shipment = 1 
    mar_WTW_gCO2e = emission_factor_WTW_gCO2TEUkm * (distancia_trayecto + detour) * teus_shipment
    mar_WTW_kgCO2e = mar_WTW_gCO2e/1000
    return mar_WTW_kgCO2e

In [21]:
# 2020 TTW CO2e, 100% UF = 59.7 grams of CO2 per TEU km
# Aplicamos 70% utilization factor
def mar_canarias_emisiones_carbono_TTW(distancia_trayecto):
    emission_factor_TTW_gCO2TEUkm = 66.8
    detour = distancia_trayecto * 0.15
    teus_shipment = 1
    mar_TTW_gCO2e = (emission_factor_TTW_gCO2TEUkm * (distancia_trayecto + detour) * teus_shipment) / 0.7
    mar_TTW_kgCO2e = mar_TTW_gCO2e/1000
    return mar_TTW_kgCO2e

In [22]:
def mar_canarias_emisiones_carbono_WTT(distancia_trayecto):
    wtw = mar_canarias_emisiones_carbono_WTW(distancia_trayecto)
    ttw = mar_canarias_emisiones_carbono_TTW(distancia_trayecto)
    wtt = wtw - ttw
    return wtt

Aplicamos las funciones.

In [23]:
# Diferentes suposiciones dependiendo del puerto de salida
canary_ports = ['TENERIFE', 'ARRECIFE', 'LA PALMA', 'LA ESTACA', 'PUERTO DEL ROSARIO', 'LA GOMERA', 'LAS PALMAS']

carga_descarga['flag'] = carga_descarga['port_name'].isin(canary_ports) * 1

In [24]:
carga_descarga['emisionesC_TTW_TEU_kgCO2e'] = carga_descarga.apply(lambda row:mar_peninsula_emisiones_carbono_TTW(row['distanceLP_km']) if row.flag==0
                                                                   else mar_canarias_emisiones_carbono_TTW(row['distanceLP_km']), axis=1)

carga_descarga['emisionesC_WTT_TEU_kgCO2e'] = carga_descarga.apply(lambda row:mar_peninsula_emisiones_carbono_WTT(row['distanceLP_km']) if row.flag==0
                                                                   else mar_canarias_emisiones_carbono_WTT(row['distanceLP_km']), axis=1)

carga_descarga['emisionesC_WTW_TEU_kgCO2e'] = carga_descarga.apply(lambda row:mar_peninsula_emisiones_carbono_WTW(row['distanceLP_km']) if row.flag==0
                                                                   else mar_canarias_emisiones_carbono_WTW(row['distanceLP_km']), axis=1)

#### 3. Trayecto descarga-llegada

##### Emisiones carbono

Aprovechamos las funciones definidas para el Tramo salida_carga.

In [25]:
descarga_llegada['emisionesC_TTW_total_kgCO2e'] = descarga_llegada.apply(lambda row:carretera_emisiones_carbono_TTW(row['consumo_diesel_total_l']), axis=1)

descarga_llegada['emisionesC_WTT_total_kgCO2e'] = descarga_llegada.apply(lambda row:carretera_emisiones_carbono_WTT(row['consumo_diesel_total_l']), axis=1)

descarga_llegada['emisionesC_WTW_total_kgCO2e'] = descarga_llegada.apply(lambda row:carretera_emisiones_carbono_WTW(row['consumo_diesel_total_l']), axis=1)

#### 4. Trayecto salida-llegada

Aprovechamos las funciones definidas para el Tramo salida_carga.

In [26]:
salida_llegada['emisionesC_TTW_total_kgCO2e'] = salida_llegada.apply(lambda row:carretera_emisiones_carbono_TTW(row['consumo_diesel_total_l']), axis=1)

salida_llegada['emisionesC_WTT_total_kgCO2e'] = salida_llegada.apply(lambda row:carretera_emisiones_carbono_WTT(row['consumo_diesel_total_l']), axis=1)

salida_llegada['emisionesC_WTW_total_kgCO2e'] = salida_llegada.apply(lambda row:carretera_emisiones_carbono_WTW(row['consumo_diesel_total_l']), axis=1)

### Subetapa 2.4: asignación de este trayecto a una porción de cada uno de los cuatro resultados de la Subetapa 2.3

- Ahora se debe asignar una parte de la energía y de las emisiones calculadas,
correspondiente a su participación relativa en la actividad de transporte ralizada
dentro del VOS.
- Ver (AENOR2013) pg. 18.
    - Primero calculamos **S(trayecto)**, el factor utilizado para calcularla proporción
    de la energía y de las emisiones del VOS que se asignan a un servicio de transporte
    para el trayecto y para el VOS asociado.
    - S(trayecto) = T(trayecto) / T(VOS)
        - T(trayecto): actividade de transporte del servicio de transport para el trayecto
        - T(VOS): actividad de transporte del VOS relativo al trayecto
- Datos cálculo: 
    - Peso total del contenedor es 12.45 toneladas (Table 9, (Schmied2012)). actual_cargo_weight_t
    - Peso de la mercancía transportada en toneladas varía. Share of diesel consumption per tonne cargo.

#### 1. Trayecto salida-carga

##### Factor proporción libros

In [27]:
# Añadir peso de los libros
salida_carga = pd.merge(salida_carga, libros[['libro_titulo', 'peso_t_500libros']], how='cross')

In [28]:
def salida_carga_factor_proporcion_carga(peso_mercancia):
    actual_cargo_weight_t = 12.45
    factor = peso_mercancia / actual_cargo_weight_t
    return factor

##### Energía total

In [29]:
def salida_carga_consumo_energetico_proporcion_TTW(peso_mercancia, consumo_diesel):
    factor = salida_carga_factor_proporcion_carga(peso_mercancia)
    total = carretera_consumo_energetico_TTW(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [30]:
def salida_carga_consumo_energetico_proporcion_WTW(peso_mercancia, consumo_diesel):
    factor = salida_carga_factor_proporcion_carga(peso_mercancia)
    total = carretera_consumo_energetico_WTW(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [31]:
def salida_carga_consumo_energetico_proporcion_WTT(peso_mercancia, consumo_diesel):
    factor = salida_carga_factor_proporcion_carga(peso_mercancia)
    total = carretera_consumo_energetico_WTT(consumo_diesel)
    proporcion = factor * total
    return proporcion

##### Emisiones carbono

In [32]:
def salida_carga_emisiones_carbono_proporcion_TTW(peso_mercancia, consumo_diesel):
    factor = salida_carga_factor_proporcion_carga(peso_mercancia)
    total = carretera_emisiones_carbono_TTW(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [33]:
def salida_carga_emisiones_carbono_proporcion_WTW(peso_mercancia, consumo_diesel):
    factor = salida_carga_factor_proporcion_carga(peso_mercancia)
    total = carretera_emisiones_carbono_WTW(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [34]:
def salida_carga_emisiones_carbono_proporcion_WTT(peso_mercancia, consumo_diesel):
    factor = salida_carga_factor_proporcion_carga(peso_mercancia)
    total = carretera_emisiones_carbono_WTT(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [35]:
# El contendor pesa en total 12.45 tonnes (actual_cargo_weight_t)
salida_carga['emisionesC_TTW_libros_kgCO2e'] = salida_carga.apply(lambda row:salida_carga_emisiones_carbono_proporcion_TTW(peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_TEU_l']), axis=1)

salida_carga['emisionesC_WTT_libros_kgCO2e'] = salida_carga.apply(lambda row:salida_carga_emisiones_carbono_proporcion_WTT(peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_TEU_l']), axis=1)

salida_carga['emisionesC_WTW_libros_kgCO2e'] = salida_carga.apply(lambda row:salida_carga_emisiones_carbono_proporcion_WTW(peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_TEU_l']), axis=1)

#### 2. Trayecto carga-descarga

##### Factor proporción libros

In [36]:
# Añadir peso de los libros
carga_descarga = pd.merge(carga_descarga, libros[['libro_titulo', 'peso_t_500libros']], how='cross')

In [37]:
# La metodología de Clean Cargo asume que 1 TEU = 10 toneladas
def mar_factor_proporcion_carga(peso_mercancia):
    actual_cargo_weight_t = 10
    factor = peso_mercancia / actual_cargo_weight_t
    return factor

##### Emisiones carbono

###### 1. Trayecto peninsular

In [38]:
def mar_peninsula_emisiones_carbono_proporcion_TTW(peso_mercancia, distancia_trayecto):
    factor = mar_factor_proporcion_carga(peso_mercancia)
    total = mar_peninsula_emisiones_carbono_TTW(distancia_trayecto)
    proporcion = factor * total
    return proporcion

In [39]:
def mar_peninsula_emisiones_carbono_proporcion_WTW(peso_mercancia, distancia_trayecto):
    factor = mar_factor_proporcion_carga(peso_mercancia)
    total = mar_peninsula_emisiones_carbono_WTW(distancia_trayecto)
    proporcion = factor * total
    return proporcion

In [40]:
def mar_peninsula_emisiones_carbono_proporcion_WTT(peso_mercancia, distancia_trayecto):
    factor = mar_factor_proporcion_carga(peso_mercancia)
    total = mar_peninsula_emisiones_carbono_WTT(distancia_trayecto)
    proporcion = factor * total
    return proporcion

###### 2. Trayecto canario

In [41]:
def mar_canarias_emisiones_carbono_proporcion_TTW(peso_mercancia, distancia_trayecto):
    factor = mar_factor_proporcion_carga(peso_mercancia)
    total = mar_canarias_emisiones_carbono_TTW(distancia_trayecto)
    proporcion = factor * total
    return proporcion

In [42]:
def mar_canarias_emisiones_carbono_proporcion_WTW(peso_mercancia, distancia_trayecto):
    factor = mar_factor_proporcion_carga(peso_mercancia)
    total = mar_canarias_emisiones_carbono_WTW(distancia_trayecto)
    proporcion = factor * total
    return proporcion

In [43]:
def mar_canarias_emisiones_carbono_proporcion_WTT(peso_mercancia, distancia_trayecto):
    factor = mar_factor_proporcion_carga(peso_mercancia)
    total = mar_canarias_emisiones_carbono_WTT(distancia_trayecto)
    proporcion = factor * total
    return proporcion

In [44]:
carga_descarga['emisionesC_TTW_libros_kgCO2e'] = carga_descarga.apply(lambda row:mar_peninsula_emisiones_carbono_proporcion_TTW(peso_mercancia=row['peso_t_500libros'], distancia_trayecto=row['distanceLP_km']) if row.flag==0
                                                                   else mar_canarias_emisiones_carbono_proporcion_TTW(peso_mercancia=row['peso_t_500libros'], distancia_trayecto=row['distanceLP_km']), axis=1)

carga_descarga['emisionesC_WTT_libros_kgCO2e'] = carga_descarga.apply(lambda row:mar_peninsula_emisiones_carbono_proporcion_WTT(peso_mercancia=row['peso_t_500libros'], distancia_trayecto=row['distanceLP_km']) if row.flag==0
                                                                   else mar_canarias_emisiones_carbono_proporcion_WTT(peso_mercancia=row['peso_t_500libros'], distancia_trayecto=row['distanceLP_km']), axis=1)

carga_descarga['emisionesC_WTW_libros_kgCO2e'] = carga_descarga.apply(lambda row:mar_peninsula_emisiones_carbono_proporcion_WTW(peso_mercancia=row['peso_t_500libros'], distancia_trayecto=row['distanceLP_km']) if row.flag==0
                                                                   else mar_canarias_emisiones_carbono_proporcion_WTW(peso_mercancia=row['peso_t_500libros'], distancia_trayecto=row['distanceLP_km']), axis=1)

#### 3. Trayecto descarga-llegada

In [45]:
# Añadir peso de los libros
descarga_llegada = pd.merge(descarga_llegada, libros[['libro_titulo', 'peso_t_500libros']], how='cross')

In [46]:
def carreteraGC_factor_proporcion_carga(peso_mercancia):
    actual_cargo_weight_t = 2.1
    factor = peso_mercancia / actual_cargo_weight_t
    return factor

##### Energía total

In [47]:
def carreteraGC_consumo_energetico_proporcion_TTW(peso_mercancia, consumo_diesel):
    factor = carreteraGC_factor_proporcion_carga(peso_mercancia)
    total = carretera_consumo_energetico_TTW(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [48]:
def carreteraGC_consumo_energetico_proporcion_WTW(peso_mercancia, consumo_diesel):
    factor = carreteraGC_factor_proporcion_carga(peso_mercancia)
    total = carretera_consumo_energetico_WTW(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [49]:
def carreteraGC_consumo_energetico_proporcion_WTT(peso_mercancia, consumo_diesel):
    factor = carreteraGC_factor_proporcion_carga(peso_mercancia)
    total = carretera_consumo_energetico_WTT(consumo_diesel)
    proporcion = factor * total
    return proporcion

##### Emisiones carbono

In [50]:
def carreteraGC_emisiones_carbono_proporcion_TTW(peso_mercancia, consumo_diesel):
    factor = carreteraGC_factor_proporcion_carga(peso_mercancia)
    total = carretera_emisiones_carbono_TTW(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [51]:
def carreteraGC_emisiones_carbono_proporcion_WTW(peso_mercancia, consumo_diesel):
    factor = carreteraGC_factor_proporcion_carga(peso_mercancia)
    total = carretera_emisiones_carbono_WTW(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [52]:
def carreteraGC_emisiones_carbono_proporcion_WTT(peso_mercancia, consumo_diesel):
    factor = carreteraGC_factor_proporcion_carga(peso_mercancia)
    total = carretera_emisiones_carbono_WTT(consumo_diesel)
    proporcion = factor * total
    return proporcion

In [53]:
descarga_llegada['emisionesC_TTW_libros_kgCO2e'] = descarga_llegada.apply(lambda row:carreteraGC_emisiones_carbono_proporcion_TTW(peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_total_l']), axis=1)

descarga_llegada['emisionesC_WTT_libros_kgCO2e'] = descarga_llegada.apply(lambda row:carreteraGC_emisiones_carbono_proporcion_WTT(peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_total_l']), axis=1)

descarga_llegada['emisionesC_WTW_libros_kgCO2e'] = descarga_llegada.apply(lambda row:carreteraGC_emisiones_carbono_proporcion_WTW(peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_total_l']), axis=1)

#### 4. Trayecto salida-llegada

In [54]:
# Añadir peso de los libros
salida_llegada = pd.merge(salida_llegada, libros[['libro_titulo', 'peso_t_500libros']], how='cross')

Se usan las funciones definidas en el Trayecto descarga-llegada.

In [55]:
salida_llegada['emisionesC_TTW_libros_kgCO2e'] = salida_llegada.apply(lambda row:carreteraGC_emisiones_carbono_proporcion_TTW(peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_total_l']), axis=1)

salida_llegada['emisionesC_WTT_libros_kgCO2e'] = salida_llegada.apply(lambda row:carreteraGC_emisiones_carbono_proporcion_WTT(peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_total_l']), axis=1)

salida_llegada['emisionesC_WTW_libros_kgCO2e'] = salida_llegada.apply(lambda row:carreteraGC_emisiones_carbono_proporcion_WTW(peso_mercancia=row['peso_t_500libros'], consumo_diesel=row['consumo_diesel_total_l']), axis=1)

In [56]:
salida_llegada_GC = salida_llegada.copy()

## Etapa 3: suma de los resultados de cada trayecto

### Puntos de inicio fuera de Gran Canaria

In [57]:
salida_carga.drop(columns=['consumo_diesel_TEU_l',
'emisionesC_TTW_TEU_kgCO2e', 'emisionesC_WTT_TEU_kgCO2e', 'emisionesC_WTW_TEU_kgCO2e'], inplace=True)

# Renombrar columnas para unir
salida_carga.rename(columns={'port_name': 'port_name_salida',
'emisionesC_TTW_libros_kgCO2e': 'emisionesC_TTW_libros_kgCO2e_SC',
'emisionesC_WTT_libros_kgCO2e': 'emisionesC_WTT_libros_kgCO2e_SC', 
'emisionesC_WTW_libros_kgCO2e': 'emisionesC_WTW_libros_kgCO2e_SC'}, inplace=True)

In [58]:
carga_descarga.drop(columns=['distanceLP_nm', 'flag',
'emisionesC_TTW_TEU_kgCO2e', 'emisionesC_WTT_TEU_kgCO2e', 'emisionesC_WTW_TEU_kgCO2e'], inplace=True)

# Renombrar columnas para unir
carga_descarga.rename(columns={'port_name': 'port_name_salida',
'emisionesC_TTW_libros_kgCO2e': 'emisionesC_TTW_libros_kgCO2e_CD',
'emisionesC_WTT_libros_kgCO2e': 'emisionesC_WTT_libros_kgCO2e_CD', 
'emisionesC_WTW_libros_kgCO2e': 'emisionesC_WTW_libros_kgCO2e_CD'}, inplace=True)

In [59]:
descarga_llegada.drop(columns=['port_latlon', 'consumo_diesel_total_l', 
'emisionesC_TTW_total_kgCO2e', 'emisionesC_WTT_total_kgCO2e', 'emisionesC_WTW_total_kgCO2e'], inplace=True)

# Renombrar columnas para unir
descarga_llegada.rename(columns={'port_name': 'port_name_llegada',
'emisionesC_TTW_libros_kgCO2e': 'emisionesC_TTW_libros_kgCO2e_DL',
'emisionesC_WTT_libros_kgCO2e': 'emisionesC_WTT_libros_kgCO2e_DL', 
'emisionesC_WTW_libros_kgCO2e': 'emisionesC_WTW_libros_kgCO2e_DL'}, inplace=True)

In [60]:
# Unir trayectos
salida_descarga_noGC = pd.merge(salida_carga, carga_descarga, on=['port_name_salida', 'libro_titulo', 'peso_t_500libros'])

salida_llegada_noGC = pd.merge(salida_descarga_noGC, descarga_llegada, on=['libro_titulo', 'peso_t_500libros'])

# Elimino los puntos de salida que son desde GC
salida_llegada_noGC = salida_llegada_noGC[salida_llegada_noGC['port_name_salida'] !='LAS PALMAS']

# Suma ruta distancia total
salida_llegada_noGC["distancia_ruta_km"] = salida_llegada_noGC['shortest_distance_to_port_km'] + salida_llegada_noGC['distanceLP_km']
+ salida_llegada_noGC['descarga_llegada_distancia_km']
# Suma ruta emisiones
salida_llegada_noGC['emisionesC_TTW_libros_kgCO2e_ruta'] = salida_llegada_noGC['emisionesC_TTW_libros_kgCO2e_SC'] + salida_llegada_noGC['emisionesC_TTW_libros_kgCO2e_CD'] + salida_llegada_noGC['emisionesC_TTW_libros_kgCO2e_DL']
salida_llegada_noGC['emisionesC_WTT_libros_kgCO2e_ruta'] = salida_llegada_noGC['emisionesC_WTT_libros_kgCO2e_SC'] + salida_llegada_noGC['emisionesC_WTT_libros_kgCO2e_CD'] + salida_llegada_noGC['emisionesC_WTT_libros_kgCO2e_DL']
salida_llegada_noGC['emisionesC_WTW_libros_kgCO2e_ruta'] = salida_llegada_noGC['emisionesC_WTW_libros_kgCO2e_SC'] + salida_llegada_noGC['emisionesC_WTW_libros_kgCO2e_CD'] + salida_llegada_noGC['emisionesC_WTW_libros_kgCO2e_DL']

### Añadir puntos de inicio en Gran Canaria

In [61]:
salida_llegada_GC.drop(columns=['consumo_diesel_total_l',
'emisionesC_TTW_total_kgCO2e', 'emisionesC_WTT_total_kgCO2e', 'emisionesC_WTW_total_kgCO2e'], inplace=True)

# Renombrar columnas para unir
salida_llegada_GC.rename(columns={'salida_llegada_distancia_km': 'distancia_ruta_km',
'emisionesC_TTW_libros_kgCO2e': 'emisionesC_TTW_libros_kgCO2e_ruta',
'emisionesC_WTT_libros_kgCO2e': 'emisionesC_WTT_libros_kgCO2e_ruta', 
'emisionesC_WTW_libros_kgCO2e': 'emisionesC_WTW_libros_kgCO2e_ruta'}, inplace=True)

Ordenar y concatenar resultados

In [62]:
salida_llegada_noGC = salida_llegada_noGC[['name', 'population', 'latlon', 'port_name_salida', 'port_latlon', 
'punto_llegada_latlon', 'libro_titulo', 'peso_t_500libros', 'distancia_ruta_km', 
'emisionesC_TTW_libros_kgCO2e_ruta', 'emisionesC_WTT_libros_kgCO2e_ruta', 'emisionesC_WTW_libros_kgCO2e_ruta']]

salida_llegada_GC = salida_llegada_GC[['name', 'population', 'latlon', 'punto_llegada_latlon', 'libro_titulo', 'peso_t_500libros',
'distancia_ruta_km', 'emisionesC_TTW_libros_kgCO2e_ruta', 'emisionesC_WTT_libros_kgCO2e_ruta', 'emisionesC_WTW_libros_kgCO2e_ruta']]

resultados_huella = pd.concat([salida_llegada_GC, salida_llegada_noGC]).reset_index(drop=True)

resultados_huella.to_csv('../data/final/resultados_huella.csv')